## Local Function Review

In [1]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
    
    return inner_function() 

In [3]:
# Test it
outer_function()

Hi


In [4]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
    # Now return the function. No ( )
    return inner_function

In [5]:
my_func = outer_function()
my_func

<function __main__.outer_function.<locals>.inner_function>

In [6]:
# Run it
my_func()

Hi


In [7]:
my_func()

Hi


In [8]:
# Now add some variables
def outer_function(msg):
    message = msg
    def inner_function():
        print(message)
    # Now return the function. No ( )
    return inner_function

In [10]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()
bye_func()

Hi
Bye


In [11]:
# Now add some variables
# Remove intermediate members
def outer_function(msg):
    def inner_function():
        print(msg)
    return inner_function

In [12]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()
bye_func()

Hi
Bye


## Decorators
- A function that takes another function as argument
- Adds some functionality
- Returns another function
- All of this without altering the original function.


In [16]:
def decorator_function(original_function):
    def wrapper_function():
        return original_function()
    # Return function
    return wrapper_function 

def display():
    print("Display function ran")

In [17]:
# Test it
decorated_display = decorator_function(display)
decorated_display

<function __main__.decorator_function.<locals>.wrapper_function>

In [18]:
decorated_display()

Display function ran


In [19]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(
        original_function.__name__))
        return original_function()
    # Return function
    return wrapper_function 

def display():
    print("Display function ran")

In [20]:
# Test it
decorated_display = decorator_function(display)
decorated_display()

Wrapper executed this before display
Display function ran


## Now decorate your functions
**@Decorator_Function_Name**

In [21]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(
        original_function.__name__))
        return original_function()
    # Return function
    return wrapper_function 

@decorator_function
def display():
    print("Display function ran")

In [22]:
# Test it
display()

Wrapper executed this before display
Display function ran


This will not work if our original function takes arguments

In [28]:
@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({},{})".format(name, age))

In [29]:
display_info("Mario", 21)

TypeError: wrapper_function() takes 0 positional arguments but 2 were given

We can fix this with **\*args** and **\*\*kwargs**

In [30]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print("Wrapper executed this before {}".format(
        original_function.__name__))
        return original_function(*args, **kwargs)
    # Return function
    return wrapper_function 

@decorator_function
def display():
    print("Display function ran")

@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({},{})".format(name, age))

In [31]:
# test it
display()

Wrapper executed this before display
Display function ran


In [33]:
display_info("Mario", 21)

Wrapper executed this before display_info
display_info ran with arguments (Mario,21)


### Example: escape unicode

In [34]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return ascii(x)
    return wrap

@escape_unicode
def mexico_city():
    # Alt + 130 for é
    return "México"

In [35]:
mexico_city()

"'M\\xe9xico'"

#### What can be a Decorator?
- Class Objects, whichcan be callable with the **dunder call**
- Functions as decorators

### Classes as Decorators

In [38]:
class DecoratorClass(object):
    def __init__(self, original_function):
        self.original_function = original_function
        
    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}".format(
        self.original_function.__name__))
        return self.original_function(*args, **kwargs)

@DecoratorClass
def display():
    print("Display function ran")

@DecoratorClass
def display_info(name, age):
    print("display_info ran with arguments ({},{})".format(name, age))

In [39]:
# test it
display()
display_info("John", 32)

Call method executed this before display
Display function ran
Call method executed this before display_info
display_info ran with arguments (John,32)


### Instances as Decorators
A class instance. 

In [40]:
class Trace:
    def __init__(self):
        self._enable = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}".format(f.__name__))
            return f(*args, **kwargs)
        return wrap
    
# Create an intance
tracer = Trace()
# Instance as decorator
@tracer
def rotate_list(l):
    return l[1:] + [l[0]]


Unlike our previous example the **class object itself is not the decorator**. Rather, instances of trace can be used as decorators. 

In [41]:
# test it
l1 = [1, 2, 3]
l1 = rotate_list(l1)

Calling rotate_list


In [42]:
l1

[2, 3, 1]

In [43]:
# turn off the enable key from 
# the instance
tracer._enable = False
l1 = [1, 2, 3]
l1 = rotate_list(l1)

### Back to function decorators

In [46]:
import time
def my_timer(original_func):
    """
    Helps you keep track of the loggin part of a function.
    """
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(original_func.__name__, t2))
        return result    
    return wrapper
# Test it
@my_timer
def display_info(name, age):
    time.sleep(2)
    print("Display_info ran with arguments ({},{})".format(
    name, age))

In [47]:
# test it
display_info("mario", 21)

Display_info ran with arguments (mario,21)
display_info ran in: 2.012935161590576


## Multiple decorators

In [50]:
def my_logger(original_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        original_func.__name__), level=logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return original_func(*args, **kwargs)
    return wrapper
    
@my_logger
def display_info(name, age):
    print("Display_info ran with arguments ({},{})".format(
    name, age))    

In [51]:
# Test it
display_info("Weber", 125)

Display_info ran with arguments (Weber,125)


In [54]:
# multiple decorators
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({},{})".format(
    name, age)) 
# Test it
display_info("mario", 21)

Display_info ran with arguments (mario,21)
wrapper ran in: 1.0146558284759521


In [55]:
# Switch order of decorators
# multiple decorators
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({},{})".format(
    name, age)) 
# Test it
display_info("mario", 21)

Display_info ran with arguments (mario,21)
display_info ran in: 1.0146710872650146


Wrap everything

In [56]:
import time
from functools import wraps

def my_logger(original_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        original_func.__name__), level=logging.INFO)
   
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return original_func(*args, **kwargs)
    return wrapper

def my_timer(original_func):
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(original_func.__name__, t2))
        return result    
    return wrapper

@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({},{})".format(
    name, age)) 

# Test it
display_info("mario", 21)

Display_info ran with arguments (mario,21)
display_info ran in: 1.0119943618774414
